In [362]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from numpy import array
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [363]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.io as pio
pio.renderers.default = "vscode"


plt.style.use('default')
%matplotlib inline

In [364]:
dataframe = pd.read_csv('clean_data.csv', usecols=[5], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [365]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(dataset)

MinMaxScaler()

In [366]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

	#dataX -> 100 200 300 Y -> 400 
	#dataY -> 200 300 400 Y -> 500 

In [367]:
#series_results = pd.DataFrame(columns = ['SVR_10', 'SVR_05', 'LSTM', 'Poly_Reg', 'ARIMA'])

# SVR(c=10)

In [368]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [369]:
#for plot
look_back = 3
data = pd.read_csv('clean_data.csv', index_col=0)
data = data.iloc[train_size+look_back+1:,:]
data.sort_index(ascending=True, inplace=True)

In [370]:
train = scaler.transform(train)
test = scaler.transform(test)

In [371]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [372]:
trainX1 = np.reshape(trainX, (-1, 3))
trainY1 = np.reshape(trainY, (917, 1))

regressor=SVR(kernel='rbf', C=10)
regressor.fit(trainX1, trainY1)

print(trainX1.shape)
print(trainY1.shape)
 

import joblib
joblib.dump(regressor, "svr_10.gz")

(917, 3)
(917, 1)


C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



['svr_10.gz']

In [373]:
pred = regressor.predict(testX)
series_results = pd.DataFrame(scaler.inverse_transform([pred]).flatten(), columns=['SVR_10'])

print(f"Mean Abs Error: {metrics.mean_absolute_error(testY, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY, pred))}")

Mean Abs Error: 0.061743450538459484
Mean Sq Error: 0.0067924560592082725
Root Mean Error: 0.08241635795889231


In [374]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform([testY]).flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform([pred]).flatten()   , name="SVR", line_shape='spline'))
fig.show()

### Cross Validation

In [375]:
from sklearn.model_selection import TimeSeriesSplit

import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

df = pd.read_csv('clean_data.csv', usecols=[5])
df.dropna(inplace=True)

tscv = TimeSeriesSplit(n_splits = 5)
rmse = []
for train_index, test_index in tscv.split(df):
    cv_train, cv_test = df.iloc[train_index], df.iloc[test_index]

    #forgotten to add look_back'ed data
    cv_trainX, cv_trainY = create_dataset(cv_train.values, look_back)
    cv_testX, cv_testY = create_dataset(cv_test.values, look_back)

    regressor.fit(cv_trainX, cv_trainY) 
    predictions = regressor.predict(cv_testX)
    true_values = cv_testY
    rmse.append(np.sqrt(mean_squared_error(true_values, predictions)))
print(rmse)
print('Avg. RMSE= ', np.mean(rmse))



[43.94330881185245, 17.786188572615377, 21.623834720957667, 15.377048014004016, 23.63852786543886]
Avg. RMSE=  24.473781596973673


In [376]:
# from sklearn.model_selection import cross_val_score
# X, Y = create_dataset(dataset, look_back)

# r = cross_val_score(regressor,X, Y, n_jobs=-1, scoring='neg_mean_squared_error')
# r_norm = [np.sqrt(q*-1) for q in r]
# print(r_norm)
# print('Avg. RMSE= ', np.mean(r_norm))


# SVR(c=5)

In [377]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [378]:
#for plot
look_back = 3
data = pd.read_csv('clean_data.csv', index_col=0)
data = data.iloc[train_size+look_back+1:,:]
data.sort_index(ascending=True, inplace=True)

In [379]:
train = scaler.transform(train)
test = scaler.transform(test)

In [380]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [381]:
trainX1 = np.reshape(trainX, (-1, 3))
trainY1 = np.reshape(trainY, (917, 1))

regressor=SVR(kernel='rbf', C=5)
regressor.fit(trainX1, trainY1)

print(trainX1.shape)
print(trainY1.shape)
 

import joblib
joblib.dump(regressor, "svr_05.gz")

(917, 3)
(917, 1)


C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



['svr_05.gz']

In [382]:
pred = regressor.predict(testX)
series_results['SVR_05'] = scaler.inverse_transform([pred]).flatten()


print(f"Mean Abs Error: {metrics.mean_absolute_error(testY, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY, pred))}")

Mean Abs Error: 0.060651779241697006
Mean Sq Error: 0.006570046859405237
Root Mean Error: 0.08105582557352209


In [383]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform([testY]).flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform([pred]).flatten()   , name="SVR", line_shape='spline'))
fig.show()

In [384]:
from sklearn.model_selection import TimeSeriesSplit

import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

df = pd.read_csv('clean_data.csv', usecols=[5])
df.dropna(inplace=True)

tscv = TimeSeriesSplit(n_splits = 5)
rmse = []
for train_index, test_index in tscv.split(df):
    cv_train, cv_test = df.iloc[train_index], df.iloc[test_index]

    #forgotten to add look_back'ed data
    cv_trainX, cv_trainY = create_dataset(cv_train.values, look_back)
    cv_testX, cv_testY = create_dataset(cv_test.values, look_back)

    regressor.fit(cv_trainX, cv_trainY) 
    predictions = regressor.predict(cv_testX)
    true_values = cv_testY
    rmse.append(np.sqrt(mean_squared_error(true_values, predictions)))
print(rmse)
print('Avg. RMSE= ', np.mean(rmse))



[43.85615464788311, 17.841788017860473, 21.58704645163328, 15.273132696670665, 23.472323855824268]
Avg. RMSE=  24.40608913397436


# Polynomial Regression with degree=10

In [385]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [386]:
#for plot
look_back = 3
data = pd.read_csv('clean_data.csv', index_col=0)
data = data.iloc[train_size+look_back+1:,:]
data.sort_index(ascending=True, inplace=True)

In [387]:
train = scaler.transform(train)
test = scaler.transform(test)

In [388]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [389]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [390]:
print(trainX.shape)
print(trainY.shape)

(917, 1, 3)
(917,)


In [391]:

from sklearn.preprocessing import PolynomialFeatures  
poly_regs = PolynomialFeatures(degree= 10)  
trainX1 = np.reshape(trainX, (-1, 3))


x_train_poly = poly_regs.fit_transform(trainX1)  
poly_lin_reg = LinearRegression()  
poly_lin_reg.fit(x_train_poly, trainY1)  

joblib.dump(poly_regs, "poly_reg.gz")
joblib.dump(poly_lin_reg, "poly_lin_reg.gz")

['poly_lin_reg.gz']

In [392]:
testX1 = np.reshape(testX, (-1, 3))
testY1 = np.reshape(testY, (450, 1))

x_test_poly = poly_regs.transform(testX1)  
pred = poly_lin_reg.predict(x_test_poly)

series_results["Poly_Reg"] = scaler.inverse_transform(pred)

series_results.index = data.index
series_results.sort_index(ascending=True, inplace=True)


print(f"Mean Abs Error: {metrics.mean_absolute_error(testY1, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY1, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY1, pred))}")

Mean Abs Error: 0.05998729541897774
Mean Sq Error: 0.009140988811850548
Root Mean Error: 0.09560851752758026


In [393]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform(testY1).flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform(pred).flatten()   , name="Polynomial reg", line_shape='spline'))
fig.show()

### Cross Validation

In [394]:
from sklearn.model_selection import TimeSeriesSplit

import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

df = pd.read_csv('clean_data.csv', usecols=[5])
df.dropna(inplace=True)

tscv = TimeSeriesSplit(n_splits = 5)
rmse = []
for train_index, test_index in tscv.split(df):
    cv_train, cv_test = df.iloc[train_index], df.iloc[test_index]

    cv_trainX, cv_trainY = create_dataset(cv_train.values, look_back)
    cv_testX, cv_testY = create_dataset(cv_test.values, look_back)

    poly_lin_reg.fit(cv_trainX, cv_trainY) 
    predictions = poly_lin_reg.predict(cv_testX)
    true_values = cv_testY
    rmse.append(np.sqrt(mean_squared_error(true_values, predictions)))
print(rmse)
print('Avg. RMSE= ', np.mean(rmse))


[41.942873274386365, 20.54895604442371, 19.28189861916761, 16.055435925162303, 23.29886870338013]
Avg. RMSE=  24.225606513304022


In [395]:
from sklearn.model_selection import cross_val_score
X, Y = create_dataset(dataset, look_back)

r = cross_val_score(poly_lin_reg,X, Y, n_jobs=-1, scoring='neg_mean_squared_error')
r_norm = [np.sqrt(q*-1) for q in r]
print(r_norm)
print('Avg. RMSE= ', np.mean(r_norm))

[42.823411873555656, 35.47385701118501, 15.052889341885912, 18.099580606059202, 23.984455160696662]
Avg. RMSE=  27.08683879867649


# ARIMA with parameters p=2, d=0, q=1

In [396]:
from statsmodels.tsa.arima.model import ARIMA

In [397]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [398]:
train = scaler.transform(train)
test = scaler.transform(test)

In [399]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [400]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [401]:
history = [x for x in train]

In [402]:
predictions = list()

In [403]:
predictions.clear()
for t in range(len(test)):
	model = ARIMA(history, order=(2,0,1))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.



predicted=0.183271, expected=0.184211
predicted=0.183512, expected=0.164474
predicted=0.179274, expected=0.177632
predicted=0.181188, expected=0.184211
predicted=0.183541, expected=0.180921
predicted=0.183912, expected=0.164474
predicted=0.179802, expected=0.171053
predicted=0.179736, expected=0.213816
predicted=0.191189, expected=0.200658


C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.193051, expected=0.184211
predicted=0.190735, expected=0.164474
predicted=0.184633, expected=0.177632
predicted=0.185148, expected=0.184211
predicted=0.186463, expected=0.180921
predicted=0.186065, expected=0.243421
predicted=0.203467, expected=0.263158
predicted=0.217591, expected=0.351974
predicted=0.251676, expected=0.276316
predicted=0.249468, expected=0.361842
predicted=0.280030, expected=0.338816
predicted=0.289777, expected=0.351974
predicted=0.303615, expected=0.335526
predicted=0.308412, expected=0.342105
predicted=0.315904, expected=0.338816
predicted=0.320407, expected=0.266447
predicted=0.301600, expected=0.319079


C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



...

predicted=0.205619, expected=0.029605


C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.151642, expected=0.243421
predicted=0.189779, expected=0.276316
predicted=0.212738, expected=0.483553
predicted=0.289415, expected=0.246711
predicted=0.258770, expected=0.430921
predicted=0.308609, expected=0.296053
predicted=0.291459, expected=0.125000
predicted=0.236246, expected=0.348684
predicted=0.275347, expected=0.171053
predicted=0.235886, expected=0.282895
predicted=0.252095, expected=0.240132
predicted=0.244022, expected=0.335526
predicted=0.269191, expected=0.240132
predicted=0.252988, expected=0.276316
predicted=0.258034, expected=0.365132
predicted=0.285736, expected=0.315789
predicted=0.285427, expected=0.315789
predicted=0.288262, expected=0.213816
predicted=0.259994, expected=0.128289
predicted=0.220667, expected=0.207237
predicted=0.221581, expected=0.226974
predicted=0.223120, expected=0.253289
predicted=0.230806, expected=0.243421
predicted=0.231666, expected=0.266447
predicted=0.239757, expected=0.289474
predicted=0.250913, expected=0.125000
predicted=0.

C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.238988, expected=0.157895
predicted=0.211280, expected=0.154605
predicted=0.197426, expected=0.125000
predicted=0.178952, expected=0.121711
predicted=0.166606, expected=0.072368
predicted=0.143301, expected=0.065789
predicted=0.127722, expected=0.072368
predicted=0.118982, expected=0.072368
predicted=0.112363, expected=0.075658
predicted=0.108645, expected=0.200658
predicted=0.143192, expected=0.121711
predicted=0.136495, expected=0.069079
predicted=0.120847, expected=0.062500
predicted=0.110942, expected=0.131579
predicted=0.124936, expected=0.082237
predicted=0.115913, expected=0.075658
predicted=0.110534, expected=0.036184
predicted=0.095284, expected=0.039474
predicted=0.087802, expected=0.059211
predicted=0.088178, expected=0.072368
predicted=0.091165, expected=0.062500
predicted=0.089500, expected=0.075658
predicted=0.092851, expected=0.098684
predicted=0.101327, expected=0.072368
predicted=0.098019, expected=0.042763
predicted=0.088369, expected=0.026316
predicted=0.

In [404]:
normal = [[val] for val in predictions]
testPredict = scaler.inverse_transform(normal[:-4])
testY2 = scaler.inverse_transform([testY])

In [405]:
print(len(testY2[0]))
print(len(testPredict))

450
450


In [406]:
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY2[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY2[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY2[0], testPredict[:,0]))}")

Mean Abs Error: 17.661360905123995
Mean Sq Error: 589.3158173644346
Root Mean Error: 24.275827840970422


In [407]:
series_results["ARIMA"] = testPredict
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform(testY1).flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=testPredict.flatten() , name="ARIMA", line_shape='spline'))
fig.show()

### Cross Validation

In [408]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

df = pd.read_csv('clean_data.csv', usecols=[5])
df.dropna(inplace=True)

tscv = TimeSeriesSplit(n_splits = 5)
rmse = []
for train_index, test_index in tscv.split(df):
    cv_train, cv_test = df.iloc[train_index], df.iloc[test_index]
    #print(cv_test.index.values[0], " ", cv_test.index.values[-1])
    model = ARIMA(cv_train.AQI, order=(2, 0, 1)).fit()
    predictions = model.predict(cv_test.index.values[0], cv_test.index.values[-1])
    true_values = cv_test.AQI
    rmse.append(np.sqrt(mean_squared_error(true_values, predictions)))
print(rmse)
print('Avg. RMSE= ', np.mean(rmse))


C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

C:\Users\Tejas\.pyenv\pyenv-win\versions\3.9.5\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.



[44.68380287269858, 32.16681138909791, 34.11015011051192, 26.308348411527017, 35.00139942332032]
Avg. RMSE=  34.45410244143115


# LSTM (Look_back=3)

In [409]:
import tensorflow as tf
tf.random.set_seed(7)

In [410]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [411]:
train = scaler.transform(train)
test = scaler.transform(test)

In [412]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [413]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [414]:
from tensorflow import keras
model = keras.models.load_model('AQI_LSTM.h5')

In [415]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

15/15 [==============================] - 0s 13ms/step


In [416]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY1 = scaler.inverse_transform([testY])

In [417]:
series_results["LSTM_3"] = testPredict
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY1[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY1[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY1[0], testPredict[:,0]))}")

Mean Abs Error: 14.568286552699073
Mean Sq Error: 420.34122893641717
Root Mean Error: 20.502224975265907


In [418]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=testY1.flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=testPredict.flatten()  , name="LSTM", line_shape='spline'))
fig.show()

### Cross Validation

In [419]:
from sklearn.model_selection import KFold

from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential

kfold = KFold(n_splits=5, shuffle=False)
print(f'trainY {np.shape(trainY)} testY {np.shape(testY)}')

inputs = np.concatenate((trainX, testX), axis=0)
targets = np.concatenate((trainY[0], testY), axis=0)
fold_no = 1
rmse = []
for train_idx, test_idx in kfold.split(inputs, targets):
    model = Sequential()

    model.add(LSTM(96, return_sequences=True, input_shape=(1, look_back),  activation='softsign', use_bias=True))
    model.add(LSTM(448, return_sequences=True, input_shape=(1, look_back), activation='softsign', use_bias=True))
    model.add(LSTM(64, return_sequences=True, input_shape=(1, look_back),  activation='softsign', use_bias=True))
    model.add(LSTM(416, return_sequences=True, input_shape=(1, look_back), activation='softsign', use_bias=True))

    model.add(Dense(1, activation='softsign'))

    model.compile(loss='mean_squared_error', optimizer='nadam', metrics=['mean_squared_error'])
    print(f'Fold #:{fold_no}')
    h = model.fit(inputs[train_idx], targets[train_idx], validation_split=0.33, epochs=10)
    # scores = model.evaluate(inputs[test_idx], targets[test_idx], use_multiprocessing=True, workers=5)
    # rmse.append(np.sqrt(scores[1]))
    mse = h.history['val_mean_squared_error'][0]
    rmse.append(np.sqrt(mse))
    print(mse)
    fold_no+=1

print(rmse)
print('Avg. RMSE= ', np.mean(rmse))

trainY (1, 917) testY (450,)
Fold #:1
Epoch 1/10
23/23 [==============================] - 7s 76ms/step - loss: 5311.4336 - mean_squared_error: 5311.4336 - val_loss: 0.1550 - val_mean_squared_error: 0.1550
Epoch 2/10
23/23 [==============================] - 1s 47ms/step - loss: 5243.0542 - mean_squared_error: 5243.0542 - val_loss: 0.4923 - val_mean_squared_error: 0.4923
Epoch 3/10
23/23 [==============================] - 1s 47ms/step - loss: 5225.7119 - mean_squared_error: 5225.7119 - val_loss: 0.5682 - val_mean_squared_error: 0.5682
Epoch 4/10
23/23 [==============================] - 1s 48ms/step - loss: 5222.2505 - mean_squared_error: 5222.2505 - val_loss: 0.5922 - val_mean_squared_error: 0.5922
Epoch 5/10
23/23 [==============================] - 1s 47ms/step - loss: 5220.9502 - mean_squared_error: 5220.9502 - val_loss: 0.6042 - val_mean_squared_error: 0.6042
Epoch 6/10
23/23 [==============================] - 1s 48ms/step - loss: 5220.2222 - mean_squared_error: 5220.2222 - val_loss: 

# LSTM (Look_back=10)

In [420]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [421]:
train = scaler.transform(train)
test = scaler.transform(test)

In [422]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [423]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [424]:
from tensorflow import keras
model = keras.models.load_model('AQI_LSTM_new.h5')

In [425]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

14/14 [==============================] - 0s 17ms/step


In [426]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY1 = scaler.inverse_transform([testY])

In [427]:
series_results["LSTM_10"] = np.append(np.zeros(look_back-3), values=testPredict)
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY1[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY1[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY1[0], testPredict[:,0]))}")

Mean Abs Error: 14.613421876821032
Mean Sq Error: 424.9508267022317
Root Mean Error: 20.61433546593806


In [428]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=testY1.flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=testPredict.flatten()  , name="LSTM_10", line_shape='spline'))
fig.show()

### Cross Validation


In [430]:
from sklearn.model_selection import KFold

from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential

kfold = KFold(n_splits=5, shuffle=False)
print(f'trainY {np.shape(trainY)} testY {np.shape(testY)}')

inputs = np.concatenate((trainX, testX), axis=0)
targets = np.concatenate((trainY[0], testY), axis=0)
fold_no = 1
rmse = []
for train_idx, test_idx in kfold.split(inputs, targets):
    model = Sequential()

    model.add(LSTM(416, return_sequences=True, input_shape=(1, look_back),  activation='softsign',  use_bias=True))
    model.add(LSTM(64, return_sequences=True, input_shape=(1, look_back), activation='softsign', use_bias=True))
    model.add(LSTM(256, return_sequences=True, input_shape=(1, look_back),  activation='softsign', use_bias=True))
    model.add(LSTM(320, return_sequences=True, input_shape=(1, look_back), activation='softsign',  use_bias=True))
    model.add(LSTM(256, return_sequences=True, input_shape=(1, look_back),  activation='softsign', use_bias=True))
    model.add(LSTM(128, return_sequences=True, input_shape=(1, look_back), activation='softsign', use_bias=True))
    model.add(LSTM(64, return_sequences=True, input_shape=(1, look_back),  activation='softsign', use_bias=True))
    model.add(LSTM(128, return_sequences=True, input_shape=(1, look_back), activation='softsign', use_bias=True))
    model.add(LSTM(256, return_sequences=True, input_shape=(1, look_back), activation='softsign', use_bias=True))
    model.add(Dense(1, activation='softsign'))

    model.compile(loss='mean_squared_error', optimizer='nadam', metrics=['mean_squared_error'])
    print(f'Fold #:{fold_no}')
    h = model.fit(inputs[train_idx], targets[train_idx], validation_split=0.33, epochs=10)
    # scores = model.evaluate(inputs[test_idx], targets[test_idx], use_multiprocessing=True, workers=5)
    # rmse.append(np.sqrt(scores[1]))
    mse = h.history['val_mean_squared_error'][0]
    rmse.append(np.sqrt(mse))
    print(mse)
    fold_no+=1

print(rmse)
print('Avg. RMSE= ', np.mean(rmse))

trainY (1, 910) testY (443,)
Fold #:1
Epoch 1/10
23/23 [==============================] - 15s 146ms/step - loss: 5184.0698 - mean_squared_error: 5184.0698 - val_loss: 0.3158 - val_mean_squared_error: 0.3158
Epoch 2/10
23/23 [==============================] - 2s 95ms/step - loss: 5106.7378 - mean_squared_error: 5106.7378 - val_loss: 0.5366 - val_mean_squared_error: 0.5366
Epoch 3/10
23/23 [==============================] - 2s 95ms/step - loss: 5099.3252 - mean_squared_error: 5099.3252 - val_loss: 0.5683 - val_mean_squared_error: 0.5683
Epoch 4/10
23/23 [==============================] - 2s 96ms/step - loss: 5097.6104 - mean_squared_error: 5097.6104 - val_loss: 0.5824 - val_mean_squared_error: 0.5824
Epoch 5/10
23/23 [==============================] - 2s 97ms/step - loss: 5096.6963 - mean_squared_error: 5096.6963 - val_loss: 0.5918 - val_mean_squared_error: 0.5918
Epoch 6/10
23/23 [==============================] - 2s 98ms/step - loss: 5096.0552 - mean_squared_error: 5096.0552 - val_loss

# Comparision of various Models

In [431]:
series_results

,SVR_10,SVR_05,Poly_Reg,ARIMA,LSTM_3,LSTM_10
SAMPLING DATE,,,,,,
2017-01-08,71.923154,70.837837,67.000000,65.714406,63.546577,0.000000
2017-02-05,69.837574,69.481984,70.562500,65.787705,63.803818,0.000000
2017-02-06,70.315271,70.072560,67.000000,64.499216,64.806122,0.000000
2017-02-11,68.973762,68.689636,65.812500,65.081110,63.386082,0.000000
2017-02-12,70.581909,69.644980,63.437496,65.796533,62.676838,0.000000
...,...,...,...,...,...,...
2021-12-02,60.647263,64.428741,59.874996,76.102289,63.809189,69.042244
2021-12-02,75.629352,73.830284,76.500000,71.048004,74.577370,77.616646
2021-12-03,65.497288,62.361467,80.062500,78.556438,76.121971,80.959763


In [432]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)
fig.update_xaxes(range=['2016-10-01', '2022-03-01'])

fig.add_trace(go.Scatter(x=series_results.index, y=testY1.flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.SVR_10  , name="SVR(c=10)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.SVR_05  , name="SVR(c=05)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.LSTM_3  , name="LSTM(3)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.LSTM_10  , name="LSTM(10)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.Poly_Reg  , name="Polynomial(10)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.ARIMA  , name="ARIMA", line_shape='spline'))


fig.show()